In [ ]:
# Restart the session afther this cell to avoid Google Colab errors
!pip install --upgrade --force-reinstall numpy==1.26.4 pandas

In [ ]:
!pip install pybibx
!pip install tabulate tqdm

In [ ]:
# Dowload .bib file
#!wget https://github.com/Valdecy/pyBibX/raw/main/assets/bibs/scopus.bib

In [ ]:
# Required Libraries
import textwrap
import pandas as pd
import bibtexparser
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase
import time
import os
import random
from multiprocessing import Pool, cpu_count
from functools import partial
import pandas as pd
import ollama

from pybibx.base import pbx_probe
from tabulate import tabulate
from utils import preprocess_text, is_non_english, published_within_last_n_years, classify_abstract, log, load_and_filter_bases
from batch import process_batch_with_retry, call_local_llm
from inclusion import apply_inclusion_filter

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Load .bib
# Arguments: file_bib = 'filename.bib'; db = 'scopus', 'wos', 'pubmed'; del_duplicated = True, False
file_name = 'dados/scopus.bib'
database  = 'scopus'
bibfile   = pbx_probe(file_bib = file_name, db = database, del_duplicated = True)

In [ ]:
print(bibfile.data.document_type.value_counts())
filtro = ['Article','Conference paper']
bibfile.data = bibfile.data[bibfile.data['document_type'].isin(filtro)]
print(bibfile.data.document_type.value_counts())

In [ ]:
file_name_acm = 'dados/acm.bib'
database_acm  = 'acm'
bibfile_acm   = pbx_probe(file_bib = file_name_acm, db = database_acm, del_duplicated = True)


In [ ]:
bibfile.merge_database(file_bib=file_name_acm, db=database_acm, del_duplicated=True)

In [ ]:
bibfile.data['year'] = bibfile.data['year'].astype(int)

In [ ]:
bibfile.data = bibfile.data[bibfile.data['year'] > 2019]

In [ ]:
# Health Analysis
health = bibfile.health_bib()

# Check Health
health

In [ ]:
print(bibfile.data['abstract'].head(2))

In [ ]:
print(bibfile.data.columns)

In [ ]:
!pip install pybtex
!pip install bibtexparser

In [ ]:
# === CONFIGURAÇÕES GLOBAIS ===
MAX_REQUESTS_PER_MINUTE = 300
SECONDS_BETWEEN_REQUESTS = 60 / MAX_REQUESTS_PER_MINUTE
BATCH_SIZE = 1
WORKERS = min(8, cpu_count())
MODELS = ["llama3:8b", "gemma3:27b-it-qat", "phi4"]
TEMPERATURE = 0

QUERY = (
    #"Does this abstract discuss artificial intelligence in feedback for learning management systems or online learning environment on education?"
    "Analyze the following scientific article abstract and determine whether it "
     "addresses the use of artificial intelligence to provide feedback in virtual learning environments.\n"
     "Consider aspects such as: the application of AI techniques, automated feedback systems, "
     "digital educational platforms, and online learning. Respond only with ‘yes’ if the article is related, "
     "or ‘no’ if it is not.\n\n"
)

def chunk_dataframe(df, batch_size: int):
    for i in range(0, len(df), batch_size):
        yield df.iloc[i : i + batch_size], i


def process_args_wrapper(args, query, model, temperature, seconds_between_requests, log_path):
    return process_batch_with_retry(*args, query=query, model=model,
                                    temperature=temperature,
                                    seconds_between_requests=seconds_between_requests,
                                    log_path=log_path)


# === PIPELINE PARA UM MODELO ===
def analyze_abstracts_parallel(
    df: pd.DataFrame,
    query: str,
    model: str,
    batch_size: int,
    workers: int,
    result_csv_path: str,
    log_path: str,
    temperature: float,
    seconds_between_requests: float,
) -> pd.DataFrame:
    if os.path.exists(result_csv_path):
        acumulado = pd.read_csv(result_csv_path)
        start = len(acumulado)
        log(f"[{model}] Retomando do índice {start}", log_path)
    else:
        acumulado = pd.DataFrame()
        start = 0

    to_process = df.iloc[start:].reset_index(drop=True)
    batches = [
        (batch, idx + start)
        for batch, idx in chunk_dataframe(to_process, batch_size)
    ]

    log(f"[{model}] Iniciando {len(batches)} lotes com {workers} workers", log_path)

    with Pool(processes=workers) as pool:
        processor = partial(
            process_args_wrapper,
            query=query,
            model=model,
            temperature=temperature,
            seconds_between_requests=seconds_between_requests,
            log_path=log_path
        )
        for outcome in pool.imap_unordered(processor, batches):
            if outcome:
                df_part = pd.DataFrame(outcome)
                acumulado = pd.concat([acumulado, df_part], ignore_index=True)
                acumulado.to_csv(result_csv_path, index=False)

    log(f"[{model}] Processamento completo.", log_path)
    return acumulado


# === RUN_ALL_MODELS MODIFICADA ===
def run_all_models(df: pd.DataFrame) -> pd.DataFrame:
    #
    #Executa o pipeline para todos os modelos em MODELS e retorna um DataFrame combinado
    #contendo todas as colunas relevant_<model>.
    #
    combined = df.copy()

    for model in MODELS:
        model_name = model.split(":")[0]
        result_path = f"temp_files/resultados_parciais_{model_name}.csv"
        log_path = f"temp_files/log_execucao_{model_name}.txt"

                # ajusta workers para alguns modelos (exemplo)
        extra = 0
        if model in ("gemma3:27b-it-qat","phi4-mini"): extra = 4
        workers = WORKERS + extra

        resultados = analyze_abstracts_parallel(
            df=combined,
            query=QUERY,
            model=model,
            batch_size=BATCH_SIZE,
            workers=workers,
            result_csv_path=result_path,
            log_path=log_path,
            temperature=TEMPERATURE,
            seconds_between_requests=SECONDS_BETWEEN_REQUESTS
        )

        col = f"relevant_{model_name}"
        combined = combined.merge(
            resultados[[col]],
            left_index=True, right_index=True
        )

    return combined


In [ ]:
import os
dados = bibfile.data
df_ieee = load_and_filter_bases("dados")
colunas_desejadas_ieee = ['Document Title', 'Abstract', 'Author Affiliations', 'Authors', 'DOI', 'ISBNs',
                             'ISSN', 'Publication Title', 'Publication Year']
df_ieee = df_ieee[colunas_desejadas_ieee].copy()
#print(df_ieee.columns)
colunas_desejadas_scopus = ['title', 'abstract', 'journal', 
                            'affiliation', 'author', 'doi', 'isbn',
                             'issn', 'year']

df_scopus = dados[colunas_desejadas_scopus].copy()
df_scopus = df_scopus.rename(columns={
    'title': 'Document Title',
    'abstract': 'Abstract',
    'abbrev_source_title': 'Publication Title',
    'affiliation': 'Author Affiliations',
    'author': 'Authors',
    'doi': 'DOI',
    'isbn': 'ISBNs',
    'issn': 'ISSN',
    'journal': 'Publication Title',
    'references': 'References',
    'url': 'URL',
    'year': 'Publication Year'
})


print(df_ieee.shape, '\t', df_scopus.shape)
dados = pd.concat([df_ieee, df_scopus], ignore_index=True)
print('Antes da remoção de duplicados: ', dados.shape)
dados.columns = dados.columns.str.lower()
dados = dados.dropna(subset=['abstract'])
dados = dados.drop_duplicates(subset=['abstract'])
dados = dados.reset_index(drop=True)
print('Após remoção de duplicados: ', dados.shape)

resultados = run_all_models(dados)

# Salvar CSV final (opcional)
resultados.to_csv("temp_files/resultados_preliminares.csv", index=False)

In [ ]:
print(resultados.columns)
# Exibir os resultados
for model in MODELS:
    model_name = model.split(":")[0]
    print(f"\nResultados para o modelo {model_name}:")
    print(resultados[f'relevant_{model_name}'].value_counts())
#dados_filtered = resultados[resultados['relevant'] != 'False']
#dados_filtered.head(3)

## Avaliação da diferença entre os modelos

In [ ]:
def apply_committee(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adiciona ao DataFrame uma coluna 'relevant' que será 'true' se a
    maioria (≥3) das colunas ['a','b','c','d','e'] for 'true', caso
    contrário 'false'.

    Parâmetros
    ----------
    df : pd.DataFrame
        DataFrame contendo as colunas 'a','b','c','d' e 'e', com valores
        'true' ou 'false' (strings).

    Retorna
    -------
    pd.DataFrame
        O mesmo DataFrame, com a coluna 'relevant' adicionada.
    """
    cols = ['relevant_llama3', 'relevant_gemma3', 'relevant_phi4']
    # Verifica se as colunas existem no DataFrame
    for col in cols:
        if col not in df.columns:
            raise ValueError(f"Coluna {col} não encontrada no DataFrame.")
    # Conta quantos 'true' por linha
    true_counts = df[cols].eq(True).sum(axis=1)
    # Define 'relevant' = 'true' se true_counts >= 3, senão 'false'
    df['relevant'] = (true_counts >= 2).map({True: True, False: False})
    return df

In [ ]:
resultados_com_relevancia = apply_committee(resultados)
resultados_com_relevancia.to_csv("temp_files/resultados_finais_com_relevancia.csv", index=False)
print(resultados_com_relevancia['relevant'].value_counts())

In [ ]:
print(resultados_com_relevancia['relevant'].value_counts())
print(resultados_com_relevancia[resultados_com_relevancia['relevant'] == True])

## Critrérios de exclusão

1. Livro
2. Artigos resumidos
3. Revisões de literatura
4. Relatórios técnicos
5. Escrito em língua estrangeira que não seja o inglês
6. Simulou cenários


In [ ]:
# ----------------------------
# Critérios de Exclusão
# ----------------------------
criterios_exclusao: dict[str, str] = {
"QE1": "Is it a Book?",
"QE2": "Is it a Summarized articles",
"QE3": "Is it a Literature reviews",
"QE4": "Is it aTechnical reports",
"QE5": "Was it written in a language other than English",
"QE6": "Is it a Simulated scenarios"
}

In [ ]:
import re
import pandas as pd
from langdetect import detect
from transformers import pipeline


# ----------------------------
# 2) Configura o zero-shot para QE1, QE2, QE3, QE4 e QE6
# ----------------------------
classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=0  # use -1 para CPU
)

# Labels sem QE5
labels = [criterios_exclusao[k] for k in criterios_exclusao if k != "QE5"]


In [ ]:
exclusion_log = "temp_files/log_exclusao.txt"
resultados_com_relevancia_apenas_True = resultados_com_relevancia[resultados_com_relevancia['relevant'] == True]
resultados_com_relevancia_apenas_True = resultados_com_relevancia_apenas_True.reset_index(drop=True)


resultados_com_relevancia_apenas_True["abstract_pp"] = resultados_com_relevancia_apenas_True["abstract"].apply(preprocess_text)

resultados_com_relevancia_apenas_True["exclude"] = resultados_com_relevancia_apenas_True["abstract_pp"].apply(is_non_english)

# 3) Aplica zero-shot apenas onde ainda não foi marcado como exclude
for idx, row in resultados_com_relevancia_apenas_True.iterrows():
    if not row["exclude"]:
        preds = classify_abstract(criterios_exclusao,classifier, labels ,row["abstract_pp"])
        if any(preds.values()):
            resultados_com_relevancia_apenas_True.at[idx, "exclude"] = True

# ----------------------------
# 4) Exibe resultado final
# ----------------------------
print(resultados_com_relevancia_apenas_True[["authors", "exclude"]])

resultados_com_relevancia_apenas_True.to_csv("temp_files/resultados_criterios_exclusao.csv", index=False)
print("Pipeline completa! Resultados em temp_files/resultados_criterios_exclusao.csv")

In [ ]:
print(resultados_com_relevancia_apenas_True[resultados_com_relevancia_apenas_True['exclude'] == False].shape)

## Critérios de inclusão

1. Forneceu feedback automatizados para os estudantes
2. Artigos primários
3. Publicado nos últimos cinco anos
4. Estudos envolvendo aprendizes em qualquer nível educacional (fundamental, médio, superior, formação corporativa) que utilizem um AVA
5. Implementações de inteligência artificial (machine learning, NLP, agentes conversacionais, sistemas especialistas, etc.) voltadas à geração de feedback automatizado
6. Ambientes virtuais de aprendizagem (Moodle, Canvas, Blackboard, Google Classroom, Open edX, entre outros)
7. Trabalhos que relatem pelo menos um dos seguintes resultados:

    7.1. Qualidade / utilidade do feedback

    7.2. Impacto no desempenho acadêmico ou engajamento

    7.3. Satisfação dos estudantes ou docentes
    
    7.4. Métricas de eficiência do sistema (tempo, custo, escalabilidade).

8. Estudos empíricos (experimentos controlados, quase‑experimentos, estudos de caso, design‑based research) e relatos de desenvolvimento avaliados (artigos de conference/journal com validação)
9. Publicações revisadas por pares: artigos de periódicos, capítulos de livro, anais de conferências.


In [ ]:
df_inclusos = resultados_com_relevancia_apenas_True[resultados_com_relevancia_apenas_True['exclude'] == False]
df_inclusos = df_inclusos.reset_index(drop=True)

In [ ]:
print(df_inclusos.columns)

In [ ]:
from datetime import datetime

# ----------------------------
# Critérios de Inclusão
# ----------------------------
criterios_inclusao: dict[int, str] = {
    1: "Provided automated feedback to students",
    2: "Primary research article",
    3: "Published in the last fifteen years",
    4: "Involves learners at any educational level using a virtual learning environment",
    5: "AI implementations aimed at automated feedback generation",
    6: "Virtual learning environments like Moodle, Canvas, Blackboard, Google Classroom, Open edX",
    7: "Reports outcomes such as quality or usefulness of feedback, academic performance or engagement impact, satisfaction, or system efficiency metrics",
    8: "Empirical study (controlled experiment, quasi-experiment, case study, design-based research)",
    9: "Peer-reviewed publication (journal article, conference paper, thesis, dissertation)"
}

# Sublabels para o critério 7 (pelo menos um deve ser atendido)
sublabels_7 = [
    "Feedback quality or usefulness",
    "Impact on academic performance or engagement",
    "Student or teacher satisfaction",
    "System efficiency metrics like time, cost, scalability"
]

# ----------------------------
# Configura zero-shot classifier
# ----------------------------
classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device=0  # use -1 para CPU
)
threshold = 0.7

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)
df_inclusos = apply_inclusion_filter(df_inclusos, classifier, criterios_inclusao, sublabels_7)
print(df_inclusos[["authors", "include"]].head(5))

# Exibe o resultado
print(df_inclusos[["authors", "publication year", "document title", "include"]])


In [ ]:
print(df_inclusos[["authors", "document title", "doi", "include"]].shape)

In [ ]:
df_inclusos.drop(columns=["abstract_pp"], inplace=True)

In [ ]:
df_inclusos.to_csv("temp_files/resultados_pos_processamento.csv", index=False, sep=",")